In [4]:
import faiss
import numpy as np
import pickle
import time

N = 11903  # Número de canciones
k = 8       # Número de vecinos más cercanos
nlist = 100  # Número de clusters
m = 16       # Cantidad de subespacios en PQ

# Cargar los datos
with open("feature_spotify.pkl", "rb") as file:
    track_ids = pickle.load(file)
    features = pickle.load(file)

# Validar que track_ids y features tengan el mismo tamaño
assert len(track_ids) == len(features), "track_ids y features no están alineados."

# Aplanar los vectores y alinear los IDs
features_flat = []
track_ids_flat = []

for idx, (track_id, feature_vectors) in enumerate(zip(track_ids, features)):
    if len(feature_vectors) > 0 and len(feature_vectors[0]) == 1280:  # Validar dimensión
        features_flat.extend(feature_vectors)  # Aplanar vectores
        track_ids_flat.extend([track_id] * len(feature_vectors))  # Repetir track_id

features_flat = np.array(features_flat, dtype="float32")  # Convertir a arreglo numpy

# Crear el índice IVFPQ
quantizer = faiss.IndexFlatL2(1280)  # Quantizador base (distancia Euclidiana)
index_ivfpq = faiss.IndexIVFPQ(quantizer, 1280, nlist, m, 8)

# Indexación
start_time = time.time()
index_ivfpq.train(features_flat)
index_ivfpq.add(features_flat)
indexation_time = time.time() - start_time
print(f"Tiempo de indexación (IVFPQ CPU): {indexation_time:.2f} segundos")

# Búsqueda K-NN
def knn_search_ivfpq(query_index, k):
    query_vector = features_flat[query_index].reshape(1, -1)
    start_time = time.time()
    D, I = index_ivfpq.search(query_vector, k)
    search_time = time.time() - start_time
    print(f"Búsqueda k-NN para el objeto en el índice {query_index} (k={k}):")
    print("Distancias:", D[0])
    print("Índices:", I[0])
    
    # Imprimir los track_ids correspondientes
    result_ids = [track_ids_flat[idx] for idx in I[0] if idx != -1]  # Ignorar índices inválidos
    print("Track IDs:", result_ids)
    print(f"Tiempo de búsqueda k-NN: {search_time:.2f} segundos")
    return result_ids, search_time

# Búsqueda por rango
def range_search_ivfpq(query_index, radius):
    query_vector = features_flat[query_index].reshape(1, -1)
    start_time = time.time()
    lims, D, I = index_ivfpq.range_search(query_vector, radius)
    range_search_time = time.time() - start_time
    print(f"Búsqueda por rango para el objeto en el índice {query_index} (radio={radius}):")
    
    # Extraer resultados por rango
    result_ids = [track_ids_flat[I[i]] for i in range(lims[0], lims[1])]
    result_distances = [D[i] for i in range(lims[0], lims[1])]
    
    for track_id, distance in zip(result_ids, result_distances):
        print(f"Track ID: {track_id}, Distancia: {distance:.2f}")
    
    print(f"Tiempo de búsqueda por rango: {range_search_time:.2f} segundos")
    return result_ids, range_search_time

# Índice de consulta
query_index = 100
rad = 5.5 * 10**6

# Ejecutar búsqueda K-NN
print()
ivfpq_res = knn_search_ivfpq(query_index, k)

# Ejecutar búsqueda por rango
print()
range_ivfpq_res = range_search_ivfpq(query_index, rad)


Tiempo de indexación (IVFPQ CPU): 21.73 segundos

Búsqueda k-NN para el objeto en el índice 100 (k=8):
Distancias: [4069372.5 5115957.5 5126740.5 5139553.  5145473.  5156438.  5166973.
 5188542. ]
Índices: [   100  59842 204240   4980  11380  27880 113460  71320]
Track IDs: ['0r0UDwngm74U5ccMQ9gMBM', '24GaspdcJWLts2R1wurDfW', '7BCrpOMsSnVpDgzQefNsyH', '3uozbmBqY6tB7dQJFtb5l5', '3ZuGavt1YrOkg6G93lgfUw', '5dGVIZ6oc0ScR4ldWGbXSU', '1BPOfdf7EwvZ6NZsiZVB9N', '5Njf3sGQoxQJqcR4gUbttZ']
Tiempo de búsqueda k-NN: 0.02 segundos

Búsqueda por rango para el objeto en el índice 100 (radio=5500000.0):
Track ID: 0r0UDwngm74U5ccMQ9gMBM, Distancia: 4069372.50
Track ID: 2GB8OypbvrvCee61FKx5dp, Distancia: 5283658.00
Track ID: 3uozbmBqY6tB7dQJFtb5l5, Distancia: 5139553.00
Track ID: 5RcZ84RmZ0TVTZQR3fWHoG, Distancia: 5462076.50
Track ID: 3ZuGavt1YrOkg6G93lgfUw, Distancia: 5145473.00
Track ID: 5UuikgHTxSRFRnC0zXx10i, Distancia: 5442923.00
Track ID: 3BzfDBrjyvLLyDnhTynkOM, Distancia: 5218200.00
Track ID: 5dGV

In [12]:
import faiss
import numpy as np
import pickle
import time

N = 11903  # Número de canciones (features)
k = 8       # Número de vecinos más cercanos

# Cargar los datos
with open("feature_spotify.pkl", "rb") as file:
    track_ids = pickle.load(file)
    features = pickle.load(file)

# Validar que track_ids y features tengan el mismo tamaño
assert len(track_ids) == len(features), "track_ids y features no están alineados."

# Aplanar los vectores y alinear los IDs
features_flat = []
track_ids_flat = []

for idx, (track_id, feature_vectors) in enumerate(zip(track_ids, features)):
    if len(feature_vectors) > 0 and len(feature_vectors[0]) == 1280:  # Validar dimensión
        features_flat.extend(feature_vectors)  # Aplanar vectores
        track_ids_flat.extend([track_id] * len(feature_vectors))  # Repetir track_id

features_flat = np.array(features_flat, dtype="float32")  # Convertir a arreglo numpy

# Crear el índice LSH
n_bits = 1024  # Número de bits para el hash LSH
index_lsh = faiss.IndexLSH(1280, n_bits)

# Indexación
start_time = time.time()
index_lsh.add(features_flat)
indexation_time = time.time() - start_time
print(f"Tiempo de indexación (LSH CPU): {indexation_time:.2f} segundos")

# Función de búsqueda K-NN
def knn_search_lsh(query_index, k):
    query_vector = features_flat[query_index].reshape(1, -1)
    start_time = time.time()
    D, I = index_lsh.search(query_vector, k)
    search_time = time.time() - start_time
    
    print(f"Búsqueda k-NN para el objeto en el índice {query_index} (k={k}):")
    print("Distancias:", D[0])
    print("Índices:", I[0])
    
    # Imprimir los track_ids correspondientes
    result_ids = [track_ids_flat[idx] for idx in I[0] if idx != -1]  # Ignorar índices inválidos
    print("Track IDs:", result_ids)
    print(f"Tiempo de búsqueda k-NN: {search_time:.2f} segundos")
    
    return result_ids, search_time

# Índice de consulta
query_index = 100

print("Buscar: ", track_ids_flat[query_index])

print()
# Ejecutar búsqueda K-NN
knn_res = knn_search_lsh(query_index, k)


Tiempo de indexación (LSH CPU): 2.17 segundos
Buscar:  0r0UDwngm74U5ccMQ9gMBM

Búsqueda k-NN para el objeto en el índice 100 (k=8):
Distancias: [  0. 224. 225. 225. 227. 233. 234. 234.]
Índices: [   100  89820  12540 234680 172300 202380 128000 173140]
Track IDs: ['0r0UDwngm74U5ccMQ9gMBM', '0onGoqbOulLD4FgxSuTYcu', '7dQeS4ErW1iWvnKdrtKwKr', '0t6DdFmFQhQYWhmfa5FJer', '398PpKV6XRMR7yH3bzMZPP', '3oQ8b6m5S531gvqZV9wEOY', '6DB2KOEwHnjkgEnBt5SdeJ', '1lx8ddGT5wCD6W2xmLeRKG']
Tiempo de búsqueda k-NN: 0.00 segundos


In [7]:
print(ivfpq_res[0])

['0r0UDwngm74U5ccMQ9gMBM', '24GaspdcJWLts2R1wurDfW', '7BCrpOMsSnVpDgzQefNsyH', '3uozbmBqY6tB7dQJFtb5l5', '3ZuGavt1YrOkg6G93lgfUw', '5dGVIZ6oc0ScR4ldWGbXSU', '1BPOfdf7EwvZ6NZsiZVB9N', '5Njf3sGQoxQJqcR4gUbttZ']


In [15]:
# comparar ids

list = []

for val in ivfpq_res[0]:
    for j in knn_res[0]:
        if val in j:
            list.append(val)

for i in list:
    print(i)

    


0r0UDwngm74U5ccMQ9gMBM
